In [13]:
import pandas as pd
import numpy as np


df = pd.read_csv('train.csv', index_col='id')

#пример данных
df.sample(5)

,gravity,ph,osmo,cond,urea,calc,target
id,,,,,,,
57,1.021,5.35,774,23.6,301,6.99,1
349,1.018,5.71,703,23.6,272,6.63,0
356,1.024,5.53,866,29.0,360,5.54,1
353,1.020,5.27,803,26.0,178,3.04,1
90,1.017,6.19,364,23.6,297,12.68,1


In [ ]:

numeric_data = df.select_dtypes(include=np.number).columns.tolist()
categoric_data = df.select_dtypes(include=['object', 'category']).columns.tolist()

print('numeric: ', *numeric_data, sep = ', ')
print('categoric: ', *categoric_data)


numeric: , gravity, ph, osmo, cond, urea, calc, target
categoric: 


В датасете нет категориальных данных, только таргет

In [27]:
#Подсчитаем количество пропусков и нулевых значений
for col in numeric_data:
    nan_count = df[col].isna().sum()
    null_count = df[col].isnull().sum()
    zero_count = (df[col] == 0).sum()
    
    print(f'{col}: Nan: {nan_count}, Null: {null_count} Zero: {zero_count}')


gravity: Nan: 0, Null: 0 Zero: 0
ph: Nan: 0, Null: 0 Zero: 0
osmo: Nan: 0, Null: 0 Zero: 0
cond: Nan: 0, Null: 0 Zero: 0
urea: Nan: 0, Null: 0 Zero: 0
calc: Nan: 0, Null: 0 Zero: 0
target: Nan: 0, Null: 0 Zero: 230


Данные не имеют пропусков и нулевых значений, нет необходимости обработки пропусков

In [ ]:
"Основные статистические показатели для числовых данных:"
print(df.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]))

          gravity          ph         osmo        cond        urea  \
count  414.000000  414.000000   414.000000  414.000000  414.000000   
mean     1.017894    5.955459   651.545894   21.437923  278.657005   
std      0.006675    0.642260   234.676567    7.514750  136.442249   
min      1.005000    4.760000   187.000000    5.100000   10.000000   
10%      1.008000    5.330000   343.600000   10.900000   95.000000   
25%      1.012000    5.530000   455.250000   15.500000  170.000000   
50%      1.018000    5.740000   679.500000   22.200000  277.000000   
75%      1.022000    6.280000   840.000000   27.600000  385.000000   
90%      1.025000    6.810000   925.200000   29.000000  443.000000   
max      1.040000    7.940000  1236.000000   38.000000  620.000000   

             calc      target  
count  414.000000  414.000000  
mean     4.114638    0.444444  
std      3.217641    0.497505  
min      0.170000    0.000000  
10%      1.053000    0.000000  
25%      1.450000    0.000000  
50%  

In [ ]:
#Добавим графики распределения данных
#Добавить графики корреляции числовых признаков между собой (если большая корреляция то часть колонок можно удалить)
#Добавить scatterplot 

Таргет является категориальной колонкой. Изучим ее


In [30]:

col = 'target'

nan_count = df[col].isna().sum()
null_count = df[col].isnull().sum()
zero_count = (df[col] == 0).sum()
unique_count = df[col].nunique()
total = df[col].count()
print(f"{col}: Total: {total} NaN: {nan_count}, Null: {null_count}, Zero: {zero_count}, Unique: {unique_count}")


target: Total: 414 NaN: 0, Null: 0, Zero: 230, Unique: 2


Все данные без пропусков, количество уникальных значений - 2, таким образом задача сводится к бинарной классификации.
Количество классов 0 - 230
Количество классов 1 - 194
Классы сбалансированы, нет необходимости в коррекции дисбаланса.